## Importando bibliotecas e o arquivo

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
df=pd.read_csv('data/Discentes_2013-2019.csv',encoding='latin1')

/home/gustavo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
valores_invalidos=[]
for i in range(len(df)):
    try:
        int(df.CD_CONCEITO_PROGRAMA[i])
    except ValueError:
        valores_invalidos.append(i)
        
#tirando os valores
df=df.drop(valores_invalidos)

#transformando em int
df.CD_CONCEITO_PROGRAMA=df.CD_CONCEITO_PROGRAMA.astype('int64')

#ajeitando o index
df.index=range(0,len(df))

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2476029 entries, 0 to 2476028
Data columns (total 22 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   AN_BASE                         int64 
 1   NM_GRANDE_AREA_CONHECIMENTO     object
 2   CD_AREA_AVALIACAO               int64 
 3   NM_AREA_AVALIACAO               object
 4   CD_ENTIDADE_CAPES               int64 
 5   SG_ENTIDADE_ENSINO              object
 6   CS_STATUS_JURIDICO              object
 7   DS_DEPENDENCIA_ADMINISTRATIVA   object
 8   CD_PROGRAMA_IES                 object
 9   NM_REGIAO                       object
 10  SG_UF_PROGRAMA                  object
 11  NM_MUNICIPIO_PROGRAMA_IES       object
 12  CD_CONCEITO_PROGRAMA            int64 
 13  NM_PAIS_NACIONALIDADE_DISCENTE  object
 14  DS_TIPO_NACIONALIDADE_DISCENTE  object
 15  TP_SEXO_DISCENTE                object
 16  AN_NASCIMENTO_DISCENTE          int64 
 17  DS_GRAU_ACADEMICO_DISCENTE      object
 18  ST

## Agrupando por PPG

In [5]:
def contar_homem(array):
    L=[]
    for i in array:
        if i=='MASCULINO':
            L.append(1)
        else:
            L.append(0)
    return L

def contar_mulher(array):
    L=[]
    for i in array:
        if i=='MASCULINO':
            L.append(0)
        else:
            L.append(1)
    return L

def contar_brasiliero(array):
    L=[]
    for i in array:
        if i=='BRASILEIRO':
            L.append(1)
        else:
            L.append(0)
    return L

def contar_estrangeiro(array):
    L=[]
    for i in array:
        if i=='BRASILEIRO':
            L.append(0)
        else:
            L.append(1)
    return L

def media_especial(array):
    '''Tira a média dos valores do array ignorando-se os zeros.'''
    valores_nao_nulos=[]
    for i in array:
        if i!=0:
            valores_nao_nulos.append(i)
    return np.mean(valores_nao_nulos)

In [6]:
df['Homens']=contar_homem(df.TP_SEXO_DISCENTE)
df['Mulheres']=contar_mulher(df.TP_SEXO_DISCENTE)
df['Brasileiros']=contar_brasiliero(df.DS_TIPO_NACIONALIDADE_DISCENTE)
df['Estrangeiros']=contar_estrangeiro(df.DS_TIPO_NACIONALIDADE_DISCENTE)
df['Quantidade_Discentes']=[1 for i in range(len(df))]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2476029 entries, 0 to 2476028
Data columns (total 27 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   AN_BASE                         int64 
 1   NM_GRANDE_AREA_CONHECIMENTO     object
 2   CD_AREA_AVALIACAO               int64 
 3   NM_AREA_AVALIACAO               object
 4   CD_ENTIDADE_CAPES               int64 
 5   SG_ENTIDADE_ENSINO              object
 6   CS_STATUS_JURIDICO              object
 7   DS_DEPENDENCIA_ADMINISTRATIVA   object
 8   CD_PROGRAMA_IES                 object
 9   NM_REGIAO                       object
 10  SG_UF_PROGRAMA                  object
 11  NM_MUNICIPIO_PROGRAMA_IES       object
 12  CD_CONCEITO_PROGRAMA            int64 
 13  NM_PAIS_NACIONALIDADE_DISCENTE  object
 14  DS_TIPO_NACIONALIDADE_DISCENTE  object
 15  TP_SEXO_DISCENTE                object
 16  AN_NASCIMENTO_DISCENTE          int64 
 17  DS_GRAU_ACADEMICO_DISCENTE      object
 18  ST

In [ ]:
df1=df.groupby(['CD_PROGRAMA_IES','AN_BASE'],as_index=False).agg({'Homens':np.sum,'Mulheres':np.sum,
                                                              'Brasileiros':np.sum, 'Estrangeiros':np.sum,
                                                             'Quantidade_Discentes':np.sum,
                                                            'QT_MES_TITULACAO':media_especial,
                                                            'CD_AREA_AVALIACAO':np.max,'CS_STATUS_JURIDICO':np.max,
                                                            'AN_NASCIMENTO_DISCENTE':np.mean,'NM_REGIAO':np.max,
                                                            'SG_UF_PROGRAMA':np.max,
                                                            'CD_CONCEITO_PROGRAMA':np.max})


#esse aviso aparece porque tem uns programas que só tem valor zero a coluna QT_MES_TITULACAO
#(zero indica que o progama ainda não foi terminado )

/home/gustavo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/gustavo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
def proporcao_homens(df):
    L=[]
    for i in range(len(df)):
        L.append(df.Homens[i]/df.Quantidade_Discentes[i])
    return L

In [ ]:
df1.insert(4,'Prop_Homens',proporcao_homens(df1),True)

In [ ]:
df1.head(10)

In [ ]:
df1.info()

## Tentando um ML

In [ ]:
#coisas para fazer o SVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm

#coisas para testar
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
df1=df1.dropna()

In [ ]:
# separando em treino e teste

x=df1.iloc[:,6:9] #hm, só três features
y=df1.iloc[:,14]

x_treino,x_teste,y_treino,y_teste=train_test_split(x,y,random_state=0,test_size=0.2)

In [ ]:
#fazendo tudo em escala 
scaler = StandardScaler().fit(x_treino)
x_treino=scaler.transform(x_treino)
x_teste=scaler.transform(x_teste)

In [ ]:
#treinando a máquina 
clf=svm.SVC()
clf.fit(x_treino,y_treino)

#testando o modelo

y_pred=clf.predict(x_teste)
print('Score F1 =',f1_score(y_teste,y_pred,average='weighted'))
print('Acurácia =',accuracy_score(y_teste,y_pred))

In [ ]:
plot_confusion_matrix(clf,x_teste,y_teste,values_format='.2g',normalize='pred',cmap='Greys')
plt.xlabel('Valores previstos')
plt.ylabel('Valores reais')
plt.title('Confusion Matrix normalizada para os valores previstos')